<a href="https://colab.research.google.com/github/anitakumar/ml/blob/main/ai_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai python-dotenv

In [2]:
# prompt: get api key- locally

import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

llm_name = 'gpt-3.5-turbo'


In [3]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


In [4]:
#create openai client

openai_client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [5]:
def generate_text_basic(prompt:str, model="gpt-3.5-turbo",system_prompt:str="You are helpfull assistant"):
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt,

            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.0,
        max_tokens=100,
    )
    return response.choices[0].message.content

In [6]:
def test_prompt(prompt:str,model="gpt-3.5-turbo",react_prompt:str=None):
    response = generate_text_basic(prompt)
    return response

In [7]:
prompt= "generate a list of 5 colors"
print(test_prompt(prompt))

1. Blue
2. Green
3. Red
4. Yellow
5. Purple


In [8]:
# try another prompt can t answer is because it has no ai agent with gpt4
print(test_prompt(prompt,model="gpt-4"))
prompt= "Should I take my umbrella out in California"
print(test_prompt(prompt))

1. Blue
2. Green
3. Red
4. Yellow
5. Purple
It depends on the specific location in California and the current weather conditions. California is a large state with diverse climates, so it's always a good idea to check the weather forecast for your specific area before deciding whether to take your umbrella with you. If there is a chance of rain or showers in the forecast, it's a good idea to bring your umbrella just in case.


In [9]:
def get_weather(location:str="California"):
  if location =="California":
    return "Sunny"
  elif location == "New York":
    return "Rainy"
  elif location == "Paris":
    return "Cloudy"
  elif  location == "London":
    return "Rainy"
  elif location == "Tokyo":
    return "Sunny"
  else:
    return "Unknown"

In [10]:
# Not getting answer you want build an ai agent
#hardcoded agent
current_weather = get_weather("California")

prompt = f""" Should I take my umbrella out when going to California based on the following weather conditions:  {current_weather}"""
print(test_prompt(prompt))




Based on the weather condition of "sunny," you may not need to take your umbrella out when going to California. However, it's always a good idea to check the weather forecast before heading out, as weather conditions can change quickly. If the forecast predicts rain or showers, then it would be wise to bring your umbrella just in case.


In [11]:
#optimize the solution
# make agent call the function automatically
# and responds

react_system_prompt ="""
You run in a loop of Thought, Action, PAUSE, Action_Response.
At the end of the loop you output an Answer.
Use Thought to understand your the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Action_Response will be the result of running those actions.
Your available actions are:
e.g get_weather: California
returns the current weather in California

Example session:

Question: What is the weather in California?
Thought: I should look up the weather in California.
Action: get_weather: California
PAUSE

You will be called again with this:

Observation: Sunny

You then output:

Action_Response: No, The weather in California is Sunny so you dont need to take an umbrella.""".strip()


In [12]:
# no hardcoded
def test_react_prompt(prompt:str,model="gpt-3.5-turbo",react_prompt:str=react_system_prompt):
    response = generate_text_basic(prompt,system_prompt=react_system_prompt)
    return response

In [13]:
prompt = """ Should I take my umbrella out when going to California"""
print(test_react_prompt(prompt,react_prompt=react_system_prompt))


Thought: I need to check the weather in California to determine if you should take an umbrella.

Action: get_weather: California
PAUSE


In [14]:
# now we need to define get_weather
#Available actions
available_actions ={
    "get_weather" : get_weather
}


In [15]:

# Change the prompt so that it returns a json
react_system_prompt ="""
ANSWER ONLY QUESTION RELATED TO WEATHER
You run in a loop of Thought, Action, PAUSE, Action_Response.
At the end of the loop you output an Answer.
Use Thought to understand your the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Action_Response will be the result of running those actions.
Your available actions are:
e.g get_weather: California
returns the current weather in California

Example session:

Question: What is the weather in California?
Thought: I should look up the weather in California.
{
  "function_name" : "get_weather",
  "function_params": {
    "city": "California"
  }
}
PAUSE

You will be called again with this:

Action_Response: Sunny

You then output:

Action_Response: No, The weather in California is Sunny so you dont need to take an umbrella.""".strip()

In [16]:
def test_react_prompt(prompt:str,model="gpt-3.5-turbo",react_prompt:str=react_system_prompt,available_actions={}):

    response = generate_text_basic(prompt,system_prompt=react_system_prompt)
    return response

In [17]:
prompt = """ Should I take my umbrella out when going to California"""
print(test_react_prompt(prompt,react_prompt=react_system_prompt))

Thought: I need to check the weather in California to see if you should take an umbrella.

{
  "function_name" : "get_weather",
  "function_params": {
    "city": "California"
  }
}

PAUSE


# Now I can call function which is json

In [18]:
import regex as re

#create helper functions
def model_to_json(model_instance):
    """
    Converts a Pydantic model instance to a JSON string.

    Args:
        model_instance (YourModel): An instance of your Pydantic model.

    Returns:
        str: A JSON string representation of the model.
    """
    return model_instance.model_dump_json()


In [19]:
def extend_search(text, span):
    # Extend the search to try to capture nested structures
    start, end = span
    nest_count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            nest_count += 1
        elif text[i] == '}':
            nest_count -= 1
            if nest_count == 0:
                return text[start:i+1]
    return text[start:end]


In [20]:
import json
def extract_json(text_response):
    pattern = r'\{.*?\}'
    matches = re.finditer(pattern, text_response, re.DOTALL)
    json_objects = []

    for match in matches:
        json_str = extend_search(text_response, match.span())
        try:
            json_obj = json.loads(json_str)
            json_objects.append(json_obj)
        except json.JSONDecodeError:
            continue
    return json_objects if json_objects else None

In [21]:
def test_react_prompt(prompt:str,model="gpt-3.5-turbo",react_prompt:str=react_system_prompt,available_actions={"get_weather":get_weather}):
    print(available_actions)
    response = generate_text_basic(prompt,system_prompt=react_system_prompt)
    json_function = extract_json(response)
    if json_function:
        function_name = json_function[0]['function_name']
        function_params = json_function[0]['function_params']
        if function_name not in available_actions:
            raise Exception(f"Unknown action: {function_name}: {function_params}")
        print(f" -- running {function_name} {function_params['city']}")
        action_function = available_actions[function_name]
        #call the function
        result = action_function(function_params['city'])
        function_result_message = f"Action_Response: {result}"
        print(function_result_message)

    print(f"Extracted json function{json_function}")

    return response

In [22]:
prompt = """ Should I take my umbrella out when going to London"""

print(test_react_prompt(prompt,react_prompt=react_system_prompt,available_actions=available_actions))


{'get_weather': <function get_weather at 0x7ec5b35ad000>}
 -- running get_weather London
Action_Response: Rainy
Extracted json function[{'function_name': 'get_weather', 'function_params': {'city': 'London'}}]
Thought: I should check the weather in London to see if you need an umbrella.

{
  "function_name" : "get_weather",
  "function_params": {
    "city": "London"
  }
}

PAUSE


In [23]:
# now we need to loop through the next action to give the result
def generate_text_with_conversation(messages, model="gpt-3.5-turbo"):
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
        max_tokens=100,
    )
    return response.choices[0].message.content

In [24]:
#our final agent
def test_react_prompt(prompt:str,model="gpt-3.5-turbo",react_prompt:str=react_system_prompt,available_actions={}):
    messages=[
        {"role":"system", "content":react_prompt},
         {"role":"user","content":prompt},

    ]
    turn_count = 1
    max_turns = 5
    while turn_count < max_turns:
      print (f"Loop:{turn_count}")
      print("----------------------")
      turn_count +=1

      response = generate_text_with_conversation(messages,model=model)
      print(response)

      json_function = extract_json(response)
      if json_function:
        function_name = json_function[0]['function_name']
        function_params = json_function[0]['function_params']
        if function_name not in available_actions:
            raise Exception(f"Unknown action: {function_name}: {function_params}")
        print(f" -- running {function_name} {function_params['city']}")
        action_function = available_actions[function_name]
        #call the function
        result = action_function(function_params['city'])
        function_result_message = f"Action_Response: {result}"
        messages.append({"role":"user", "content":function_result_message})
        print(function_result_message)

      else:
        break


In [25]:
prompt=""" Should I take my umbrella out when going to London"""
test_react_prompt(prompt, model="gpt-3.5-turbo",react_prompt=react_system_prompt,available_actions=available_actions)

Loop:1
----------------------
Thought: I need to check the weather in London to see if I should take an umbrella.

{
  "function_name" : "get_weather",
  "function_params": {
    "city": "London"
  }
}

PAUSE
 -- running get_weather London
Action_Response: Rainy
Loop:2
----------------------
No, you should take your umbrella out when going to London as the weather is rainy.


In [26]:
# Use simpllerLLM
!pip install simplerLLM


In [27]:
from SimplerLLM.language.llm import LLM, LLMProvider
from SimplerLLM.tools.json_helpers import extract_json_from_text

In [28]:
llm_instance= LLM.create(provider=LLMProvider.OPENAI,model_name='gpt-3.5-turbo')

In [29]:
def test_react_prompt(prompt:str,react_prompt:str=react_system_prompt,available_actions={}):
    messages=[
        {"role":"system", "content":react_prompt},
         {"role":"user","content":prompt},

    ]
    turn_count = 1
    max_turns = 5
    while turn_count < max_turns:
      print (f"Loop:{turn_count}")
      print("----------------------")
      turn_count +=1

      response = llm_instance.generate_response(messages=messages)
      print(response)
      response= str(response)

      json_function = extract_json_from_text(response)
      if json_function:
        function_name = json_function[0]['function_name']
        function_params = json_function[0]['function_params']
        if function_name not in available_actions:
            raise Exception(f"Unknown action: {function_name}: {function_params}")
        print(f" -- running {function_name} {function_params}")
        action_function = available_actions[function_name]
        #call the function
        result = action_function(**function_params)
        function_result_message = f"Action_Response: {result}"
        messages.append({"role":"user", "content":function_result_message})
        print(function_result_message)

      else:
        break


In [30]:
prompt=""" Should I take my umbrella out when going to London"""
test_react_prompt(prompt,react_prompt=react_system_prompt,available_actions=available_actions)

Loop:1
----------------------
Failed after 3 attempts due to: Connection error.
None
